In [1]:
from collections import Counter
from tqdm import tqdm
import numpy as np
import pandas as pd
import  math      
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
from geopy.distance import geodesic as GD
import folium
import os
import random
import networkx as nx
import time
from pre_processing.road_network_route import load_rn_shp_route
import ast
save_path = './Trajectory_Reconstruction/data_experiment'
E_data_name = 'Porto'
data_path_raw = '{}/{}/RN_model_data/roadnet_raw'.format(save_path,E_data_name)
data_path = '{}/{}/RN_model_data/roadnet'.format(save_path,E_data_name)
save_path_map = '{}/{}'.format(save_path,E_data_name)

# 1  Delete duplicate edges inside the file

In [2]:
def process_edgeOSM(input_file, output_file):
    unique_edges = {}  
    line_mapping = {}  
    new2raw_mapping = {}

    with open(input_file, 'r') as file:
        for idx,line in enumerate(file):
            data = line.strip().split('\t')

            edge_id = int(data[0])
            edge_start = int(data[1])
            edge_end = int(data[2])

            coords_count = int(data[3])
            start_latitude,start_longitude = float(data[4]), float(data[5])
            end_longitude, end_latitude = float(data[-1]), float(data[-2])

            coords_str = "\t".join(data[4:])
         
            edge_key = ((start_latitude,start_longitude),(end_latitude,end_longitude))
            if edge_key not in unique_edges.keys():
                unique_edges[edge_key] = (len(unique_edges), edge_start, edge_end, coords_count, coords_str,idx)
                line_mapping[idx] = len(unique_edges)-1
                new2raw_mapping[len(unique_edges)-1] = idx
            else:
                line_mapping[idx] = line_mapping[unique_edges[edge_key][-1]]
                if coords_count > int(unique_edges[edge_key][3]):
                    new2raw_mapping[line_mapping[unique_edges[edge_key][-1]]] = idx
                    unique_edges[edge_key] =  (line_mapping[unique_edges[edge_key][-1]], edge_start, edge_end, coords_count, coords_str,idx)
                    
    
    unique_edges = dict(sorted(unique_edges.items(), key=lambda item: item[1][0])) # eid  from 0 to big
 
    with open(output_file, 'w') as outfile:
   
        for edge_data in unique_edges.values():
            outfile.write("\t".join(str(item) for item in edge_data[:-1]) + "\n")

    return line_mapping,unique_edges,new2raw_mapping



input_file = '{}/edgeOSM.txt'.format(data_path_raw)
output_file = '{}/edgeOSM.txt'.format(data_path)
line_mapping,unique_edges,new2raw_mapping = process_edgeOSM(input_file,output_file)

In [3]:
len(Counter(unique_edges.keys()))

39935

In [4]:
len(line_mapping)

40267

In [5]:
len(new2raw_mapping)

39935

In [6]:
def filter_lines_by_mapping(input_file, output_file, mapping_values):

    with open(input_file, 'r') as file:
        way_data = []
        for idx,line in enumerate(file):
            data = line.strip().split('\t')
            if idx in new2raw_mapping.values():
                data[0] = line_mapping[idx]
                way_data.append(data)
    way_data = sorted(way_data, key=lambda item: item[0]) # eid  from 0 to big
    with open(output_file, 'w') as outfile:
     
        for data in way_data:
            outfile.write("\t".join(str(item) for item in data) + "\n")
    return way_data
# 主程序
wayTypeOSM_file = "{}/wayTypeOSM.txt".format(data_path_raw)
output_file = "{}/wayTypeOSM.txt".format(data_path)
way_data = filter_lines_by_mapping(wayTypeOSM_file, output_file, new2raw_mapping)

In [7]:
len(way_data)

39935

# 2 Get the graph structure with edges as nodes

In [2]:
way_type = []
features = []
lat_lng = []
edgeNum = 0

wayFile = open(data_path + '/wayTypeOSM.txt')
for line in wayFile.readlines():
    item_list = line.strip().split()
    roadId = int(item_list[0])
    wayId = int(item_list[-1])
    way_type.append(wayId)
print('way type number',len(Counter(way_type)))
edgeFile = open(data_path + '/edgeOSM.txt')
for line in edgeFile.readlines():
    item_list = line.strip().split()
    u_id = int(item_list[1])
    v_id = int(item_list[2])  # frome u_id to v_id

    single_lat_lng = []
    num = int(item_list[3])
    for i in range(num):
        tmplat = float(item_list[4 + i * 2])
        tmplon = float(item_list[5 + i * 2])
        single_lat_lng.append((float(tmplat),float(tmplon)))
    lat_lng.append(single_lat_lng)
    
    # Calculate the distance using geodesic  wrong count method  , This method is a two-point distance, not a road network distance.
    # lat1, lon1 = single_lat_lng[0]
    # lat2, lon2 = single_lat_lng[-1]
    # length = GD((lat1, lon1), (lat2, lon2)).m

    features.append([way_type[edgeNum]])
    edgeNum+=1 

way type number 7


In [3]:
len(features)

39935

In [4]:
# is constructed as a directed graph
def create_graph(nodes, node_attrs={},  is_directed=True):
    g = nx.DiGraph()
    # add nodes
    for node in nodes:
        g.add_node(node, **node_attrs.get(node, node_attrs))
    return g
# example usage
nodes = [ i for i in range(edgeNum)]
node_attrs = {s_node:{'eid': i+1,'zone_area':((np.min([sv[1] for sv in lat_lng[i]]),np.min([ sv[0] for sv in lat_lng[i]]),np.max([ sv[1] for sv in lat_lng[i]]),np.max([ sv[0] for sv in lat_lng[i]]))),'coords':lat_lng[i],"features":features[i]} for i,s_node in enumerate(nodes)}
edge_g = create_graph(nodes, node_attrs=node_attrs)

In [5]:
nx.write_gpickle(edge_g, '{}/{}_graph.gpickle'.format(save_path_map,E_data_name))
edge_g = nx.read_gpickle('{}/{}_graph.gpickle'.format(save_path_map,E_data_name))

In [6]:
len(edge_g.nodes()) #  39935

39935

In [7]:
lat = [ single_value[0] for value in lat_lng for single_value in value]
lng = [ single_value[1] for value in lat_lng for single_value in value]
print('{}:zone_range = [{}, {}, {}, {}]'.format(E_data_name,min(lat),min(lng),max(lat),max(lng)))

Porto:zone_range = [41.0876277, -8.7056464, 41.2103117, -8.4685492]


# 3 Get the graph structure with nodes as nodes

In [10]:
# change the graph to the type we want
import networkx as nx
def read_node_data(file_path):
    nodes = {}
    with open(file_path, 'r') as file:
        for line in file:
            node_id, latitude, longitude = line.strip().split('\t')
            nodes[int(node_id)] = (float(longitude), float(latitude))
    return nodes
aa_list = []
def read_edge_data(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split('\t')
            edge_id = int(data[0])
            start_node_id = int(data[1])
            end_node_id = int(data[2])
            start_latitude,start_longitude = float(data[4]), float(data[5])
            end_longitude, end_latitude = float(data[-1]), float(data[-2])
            coordinates = [(float(data[i + 4]), float(data[i + 5])) for i in range(0, len(data[4:]), 2)]
            aa_list.append((start_node_id,end_node_id))
            # aa_list.append(((start_longitude, start_latitude), (end_longitude, end_latitude)))
            edges.append((edge_id, (start_longitude, start_latitude), (end_longitude, end_latitude),start_node_id, end_node_id,coordinates))
    return edges
def create_graph(nodes, edges):
    g = nx.DiGraph()
    for node_id, (longitude, latitude) in nodes.items():
        osmid = {'osmid':node_id}
        g.add_node((longitude, latitude), **osmid)

    for edge_id, start_coords, end_coords,start_node_id,end_node_id, coordinates in edges:
        edge_data = {
            'u': start_node_id,
            'v': end_node_id,
            'eid':edge_id+1, 
            'key': edge_id,
            'coords':node_attrs[edge_id]['coords'],
            'zone_area':node_attrs[edge_id]['zone_area'],
            'Wkt': 'LINESTRING (' + ','.join(f'{lon} {lat}' for lon, lat in coordinates) + ')'
        }
        g.add_edge(start_coords, end_coords, **edge_data)

    return g

# read nodeOSM.txt,edgeOSM.txt
node_file_path = '{}/nodeOSM.txt'.format(data_path)
edge_file_path = '{}/edgeOSM.txt'.format(data_path)
nodes_data = read_node_data(node_file_path)
edges_data = read_edge_data(edge_file_path)


graph = create_graph(nodes_data, edges_data)
nx.write_gpickle(graph, '{}/{}_route_graph.gpickle'.format(save_path_map,E_data_name))
graph = nx.read_gpickle('{}/{}_route_graph.gpickle'.format(save_path_map,E_data_name))

In [13]:
len(graph.nodes())

18154

In [14]:
len(graph.edges())

39935

# 4 Initial processing and saving of trajectory data

In [23]:
traj_data = pd.read_csv('{}/Porto/RN_model_data/traj_raw/train.csv'.format(save_path))
traj_data= traj_data[traj_data['MISSING_DATA'] != True]
traj_data = traj_data[['TRIP_ID','TAXI_ID','TIMESTAMP','POLYLINE']]
traj_data.head()

,TRIP_ID,TAXI_ID,TIMESTAMP,POLYLINE
0,1372636858620000589,20000589,1372636858,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,20000596,1372637303,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,20000320,1372636951,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,20000520,1372636854,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,20000337,1372637091,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [24]:
len(Counter(traj_data['TAXI_ID']))

448

In [4]:
# 'timestamp','latitude','longitude','speed','uid'

In [25]:
def split_polyline_and_timestamp(row):
    polyline_list = ast.literal_eval(row['POLYLINE'])
    longitude_list = [point[0] for point in polyline_list]
    latitude_list = [point[1] for point in polyline_list]
    timestamps = [row['TIMESTAMP']]
    for i in range(1, len(polyline_list)):
        timestamps.append(int(timestamps[-1]) + 15)
    return pd.Series({'longitude': longitude_list, 'latitude': latitude_list, 'timestamp': timestamps})

traj_data[['longitude', 'latitude', 'timestamp']] = traj_data.apply(split_polyline_and_timestamp, axis=1)

traj_data.drop(columns=['POLYLINE','TIMESTAMP'], inplace=True)
traj_data = traj_data[traj_data['longitude'].apply(lambda x: len(x)>0)]
traj_data = traj_data.reset_index(drop=True)

In [26]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate distance between two lat/lon points
    from math import radians, sin, cos, sqrt, atan2

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    R = 6371.0  # Earth's radius in km

    # Element-wise calculations for dlat and dlon
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance
# Calculate speed
def calculate_speed(dataframe):
    # Calculate time difference between consecutive timestamps in seconds
    time_diff = (dataframe['timestamp'] - dataframe['timestamp'].shift(1))
    # Calculate distance between consecutive latitudes/longitudes in km
    distance = calculate_distance(dataframe['latitude'], dataframe['longitude'],
                                  dataframe['latitude'].shift(1), dataframe['longitude'].shift(1))
    # Calculate speed in km/s
    speed = distance / time_diff
    return speed

In [ ]:
traj_dict = defaultdict()
b =Counter(traj_data['TAXI_ID'])
b = dict(sorted(b.items()))
code_dict = {key:i for i,key in enumerate(b.keys())}

for i in tqdm(range(len(traj_data))):
    df = pd.DataFrame(traj_data.iloc[i]).T
    df_expanded = df.explode(['longitude', 'latitude', 'timestamp'], ignore_index=True)
    df_expanded['longitude'] = df_expanded['longitude'].astype(float).apply(pd.to_numeric)
    df_expanded['latitude'] = df_expanded['latitude'].astype(float).apply(pd.to_numeric)
    df_expanded['timestamp'] = df_expanded['timestamp'].astype(int).apply(pd.to_numeric)
    
    df_expanded['TAXI_ID'] = df_expanded['TAXI_ID'].map(code_dict)
    df_expanded['speed'] = calculate_speed(df_expanded)
    df_expanded['speed'].fillna(0, inplace=True)

    traj_dict[df_expanded['TRIP_ID'][0]] = df_expanded

In [ ]:
# 2.3 Longitude and latitude acquisition, latitude and longitude coordinate conversion

In [ ]:
traj_dict_all = {key: value for key, value in traj_dict.items() if len(traj_dict[key]) != 1}
np.save('{}/traj_dict_all.npy'.format(save_path_map),traj_dict_all) # 注意带上后缀名
print('save well')

# 5 get traj_dict in zone_area  and given length

In [2]:
traj_dict_all= np.load('{}/traj_dict_all.npy'.format(save_path_map),allow_pickle=True).item()

In [3]:
traj_in_area = [41.121988, -8.667057, 41.177464, -8.585300]
min_latitude, min_longitude, max_latitude, max_longitude = traj_in_area
traj_dict_in_area = {}
for key in tqdm(traj_dict_all.keys()):
    latitude_check = (traj_dict_all[key]['latitude'] >= min_latitude) & (traj_dict_all[key]['latitude'] <= max_latitude)
    longitude_check = (traj_dict_all[key]['longitude'] >= min_longitude) & (traj_dict_all[key]['longitude'] <= max_longitude)
    if latitude_check.all() and longitude_check.all():
        try:
            traj_dict_all[key].rename(columns={'TAXI_ID': 'uid'}, inplace=True)
            traj_dict_all[key]['timestamp_long'] =pd.to_datetime(traj_dict_all[key]['timestamp'], unit='s')
        except:
            pass
        traj_dict_in_area[key] = traj_dict_all[key]

100%|██████████| 1674141/1674141 [52:10<00:00, 534.81it/s]  


In [5]:
print(len(traj_dict_in_area))
traj_value = [traj_dict_in_area[key] for key in traj_dict_in_area.keys()]
length = [len(value) for value in traj_value]
count_num = len([num for num in length if num >60])
print(count_num)
print(np.mean(length))

908938
119017
40.25688770851257


In [10]:
len(traj_dict_in_area)

119017

In [11]:
np.save('{}/traj_dict.npy'.format(save_path_map),traj_dict_in_area) # 注意带上后缀名
print('save well')
# traj_dict_in_area= np.load('{}/traj_dict.npy'.format(save_path_map),allow_pickle=True).item()
# print(len(traj_dict_in_area))

save well


# 6 generate traj_dict_test

In [12]:
traj_dict= np.load('{}/traj_dict.npy'.format(save_path_map),allow_pickle=True).item()
print(len(traj_dict))
keys_list= list(traj_dict.keys())[:10]
traj_dict_test = {key:traj_dict[key] for key in keys_list}
len(traj_dict_test)
np.save('{}/traj_dict_test.npy'.format(save_path_map),traj_dict_test) # 注意带上后缀名

119017


# 7 split data

In [13]:
def split_traj(traj_dict,traj_dict_name):

    total_keys = len(traj_dict)
    sub_dict_size = total_keys // 10  

    sub_dicts = []
    for i in range(0, total_keys, sub_dict_size):
        sub_dict = {k: traj_dict[k] for k in list(traj_dict.keys())[i:i + sub_dict_size]}
        sub_dicts.append(sub_dict)
    
    for i in tqdm(range(len(sub_dicts))):
        traj_dict_split = sub_dicts[i]
        np.save('{}/{}_{}.npy'.format(save_path_map,traj_dict_name,i),traj_dict_split) # 注意带上后缀名

    print('split well')

In [14]:
traj_dict = np.load('{}/traj_dict.npy'.format(save_path_map),allow_pickle=True).item()
split_traj(traj_dict,'traj_dict')

100%|██████████| 11/11 [00:25<00:00,  2.29s/it]

split well
